# Code for the project:

## BioVision-X: Efficient Bacterial Classification in Microscopy Imaging using CNN-Transformer Hybrid Models

### Architecture: EfficientNet B0 + Transformer Blocks
### Data: Original + Cross-Validation

#### Loading libraries and Data Prep

In [1]:
!pip install --pre torch
!pip install xformers pytorch_lightning numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import torch.nn as nn
from torchvision.models import mobilenet_v2
from xformers.components.attention import ScaledDotProduct
from xformers.components.feedforward import MLP
from einops import rearrange
import numbers
from xformers.components import build_attention, feedforward
import torch.nn.functional as F


<ipython-input-2-b9046d3babbd>:4: FutureWarning: xformers.components is deprecated and is not maintained anymore. It might be removed in a future version of xFormers 
  from xformers.components.attention import ScaledDotProduct


In [3]:
!wget https://raw.githubusercontent.com/gallardorafael/EfficientMobileDL_Bacterial/refs/heads/main/Notebooks/scripts/utils.py
!mkdir scripts
!mv utils.py scripts/

--2024-11-15 06:19:43--  https://raw.githubusercontent.com/gallardorafael/EfficientMobileDL_Bacterial/refs/heads/main/Notebooks/scripts/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4120 (4.0K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   4.02K  --.-KB/s    in 0s      

2024-11-15 06:19:43 (71.9 MB/s) - ‘utils.py’ saved [4120/4120]



In [ ]:
!pip install pytorch-model-summary

In [5]:
!mkdir Correct_DIBAS
%cd /content/Correct_DIBAS

/content/Correct_DIBAS


In [6]:
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Acinetobacter.baumanii.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Actinomyces.israeli.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Bacteroides.fragilis.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Bifidobacterium.spp.zip --no-check-certificate
# !wget https://doctoral.matinf.uj.edu.pl/database/dibas/Candida.albicans.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Clostridium.perfringens.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Enterococcus.faecium.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Enterococcus.faecalis.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Escherichia.coli.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Fusobacterium.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Lactobacillus.casei.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Lactobacillus.crispatus.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Lactobacillus.delbrueckii.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Lactobacillus.gasseri.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Lactobacillus.jehnsenii.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Lactobacillus.johnsonii.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Lactobacillus.paracasei.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Lactobacillus.plantarum.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Lactobacillus.reuteri.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Lactobacillus.rhamnosus.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Lactobacillus.salivarius.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Listeria.monocytogenes.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Micrococcus.spp.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Neisseria.gonorrhoeae.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Porfyromonas.gingivalis.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Propionibacterium.acnes.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Proteus.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Pseudomonas.aeruginosa.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Staphylococcus.aureus.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Staphylococcus.epidermidis.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Staphylococcus.saprophiticus.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Streptococcus.agalactiae.zip --no-check-certificate
!wget https://doctoral.matinf.uj.edu.pl/database/dibas/Veionella.zip --no-check-certificate

--2024-11-15 06:19:49--  https://doctoral.matinf.uj.edu.pl/database/dibas/Acinetobacter.baumanii.zip
Resolving doctoral.matinf.uj.edu.pl (doctoral.matinf.uj.edu.pl)... 149.156.65.236
Connecting to doctoral.matinf.uj.edu.pl (doctoral.matinf.uj.edu.pl)|149.156.65.236|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 124685064 (119M) [application/zip]
Saving to: ‘Acinetobacter.baumanii.zip’

Acinetobacter.bauma 100%[===================>] 118.91M  10.1MB/s    in 13s     

2024-11-15 06:20:04 (8.93 MB/s) - ‘Acinetobacter.baumanii.zip’ saved [124685064/124685064]

--2024-11-15 06:20:04--  https://doctoral.matinf.uj.edu.pl/database/dibas/Actinomyces.israeli.zip
Resolving doctoral.matinf.uj.edu.pl (doctoral.matinf.uj.edu.pl)... 149.156.65.236
Connecting to doctoral.matinf.uj.edu.pl (doctoral.matinf.uj.edu.pl)|149.156.65.236|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting

In [7]:
%cd Correct_DIBAS/

[Errno 2] No such file or directory: 'Correct_DIBAS/'
/content/Correct_DIBAS


In [8]:
! unzip Acinetobacter.baumanii.zip -d Acinetobacter.baumanii
! unzip Lactobacillus.delbrueckii.zip -d Lactobacillus.delbrueckii
! unzip Neisseria.gonorrhoeae.zip -d Neisseria.gonorrhoeae
! unzip Actinomyces.israeli.zip -d Actinomyces.israeli
! unzip Lactobacillus.gasseri.zip -d Lactobacillus.gasseri
! unzip Porfyromonas.gingivalis.zip -d Porfyromonas.gingivalis
! unzip Bacteroides.fragilis.zip -d Bacteroides.fragilis
! unzip Lactobacillus.jehnsenii.zip -d Lactobacillus.jehnsenii
! unzip Propionibacterium.acnes.zip -d Propionibacterium.acnes
! unzip Bifidobacterium.spp.zip -d Bifidobacterium.spp
! unzip Lactobacillus.johnsonii.zip -d Lactobacillus.johnsonii
! unzip Proteus.zip -d Proteus
! unzip Clostridium.perfringens.zip -d Clostridium.perfringens
! unzip Lactobacillus.paracasei.zip -d Lactobacillus.paracasei
! unzip Pseudomonas.aeruginosa.zip -d Pseudomonas.aeruginosa
! unzip Enterococcus.faecalis.zip -d Enterococcus.faecalis
! unzip Lactobacillus.plantarum.zip -d Lactobacillus.plantarum
! unzip Staphylococcus.aureus.zip -d Staphylococcus.aureus
! unzip Enterococcus.faecium.zip -d Enterococcus.faecium
! unzip Lactobacillus.reuteri.zip -d Lactobacillus.reuteri
! unzip Staphylococcus.epidermidis.zip -d Staphylococcus.epidermidis
! unzip Escherichia.coli.zip -d Escherichia.coli
! unzip Lactobacillus.rhamnosus.zip -d Lactobacillus.rhamnosus
! unzip Staphylococcus.saprophiticus.zip -d Staphylococcus.saprophiticus
! unzip Fusobacterium.zip -d Fusobacterium
! unzip Lactobacillus.salivarius.zip -d Lactobacillus.salivarius
! unzip Streptococcus.agalactiae.zip -d Streptococcus.agalactiae
! unzip Lactobacillus.casei.zip -d Lactobacillus.casei
! unzip Listeria.monocytogenes.zip -d Listeria.monocytogenes
! unzip Veionella.zip -d Veionella
! unzip Lactobacillus.crispatus.zip -d Lactobacillus.crispatus
! unzip Micrococcus.spp.zip -d Micrococcus.spp

Archive:  Acinetobacter.baumanii.zip
   creating: Acinetobacter.baumanii/Acinetobacter.baumanii/
  inflating: Acinetobacter.baumanii/Acinetobacter.baumanii/Acinetobacter.baumanii_0001.tif  
  inflating: Acinetobacter.baumanii/Acinetobacter.baumanii/Acinetobacter.baumanii_0002.tif  
  inflating: Acinetobacter.baumanii/Acinetobacter.baumanii/Acinetobacter.baumanii_0003.tif  
  inflating: Acinetobacter.baumanii/Acinetobacter.baumanii/Acinetobacter.baumanii_0004.tif  
  inflating: Acinetobacter.baumanii/Acinetobacter.baumanii/Acinetobacter.baumanii_0005.tif  
  inflating: Acinetobacter.baumanii/Acinetobacter.baumanii/Acinetobacter.baumanii_0006.tif  
  inflating: Acinetobacter.baumanii/Acinetobacter.baumanii/Acinetobacter.baumanii_0007.tif  
  inflating: Acinetobacter.baumanii/Acinetobacter.baumanii/Acinetobacter.baumanii_0008.tif  
  inflating: Acinetobacter.baumanii/Acinetobacter.baumanii/Acinetobacter.baumanii_0009.tif  
  inflating: Acinetobacter.baumanii/Acinetobacter.baumanii/Acineto

In [9]:
%cd ..

/content


In [10]:
# Imports here
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import torch.nn.functional as F
import pandas as pd
import time
import numpy as np

from PIL import Image
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import KFold
from torch import nn
from torch import optim
from torch.utils.data import SubsetRandomSampler
from torchvision import datasets, transforms, models
from pytorch_model_summary import summary

# # Archs not in Pytorch
# from efficientnet_pytorch import EfficientNet

# External functions
from scripts.utils import *

In [11]:
print(torch.__version__)

import sys
print(sys.version)

2.5.1+cu124
3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]


## Data paths and hyperparameters

#### Hyperparameters and dataset details.

In [12]:
# Dataset details
dataset_version = 'original' # original or augmented
img_shape = (224,224)
img_size = str(img_shape[0])+"x"+str(img_shape[1])

# Root directory of dataset
data_dir = 'Correct_DIBAS/'

train_batch_size = 32
val_test_batch_size = 32
feature_extract = False
pretrained = True
h_epochs = 15
kfolds = 10

## Data preparation and loading

#### Deleting Corrupted files

In [15]:
# Check for corrupted TIF files

for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file == '.ipynb_checkpoints':
            continue
        if file.endswith('.tif'):
            try:
                image = Image.open(os.path.join(root, file)) # open the image file
                image.verify() # verify that it is, in fact an image

            except (IOError, SyntaxError, ValueError) as e:
                print('Bad file:', os.path.join(root, file)) # print out the names of corrupt files


In [14]:
! rm -rf Correct_DIBAS/Micrococcus.spp/Micrococcus.spp_0021.tif
! rm -rf Correct_DIBAS/Micrococcus.spp/Micrococcus.spp_0023.tif
! rm -rf Correct_DIBAS/Listeria.monocytogenes/Listeria.monocytogenes_0023.tif

#### Defining transforms and creating dataloaders

In [16]:
# Define transforms for input data
training_transforms = transforms.Compose([transforms.Resize((224,224), Image.LANCZOS),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])

total_set = datasets.ImageFolder(data_dir, transform=training_transforms)

# Defining folds
splits = KFold(n_splits = kfolds, shuffle = True, random_state = 42)

#### Visualizing the target classes in dataset

In [17]:
train_labels = {value : key for (key, value) in total_set.class_to_idx.items()}

print(len(train_labels))
print(train_labels)

32
{0: 'Acinetobacter.baumanii', 1: 'Actinomyces.israeli', 2: 'Bacteroides.fragilis', 3: 'Bifidobacterium.spp', 4: 'Clostridium.perfringens', 5: 'Enterococcus.faecalis', 6: 'Enterococcus.faecium', 7: 'Escherichia.coli', 8: 'Fusobacterium', 9: 'Lactobacillus.casei', 10: 'Lactobacillus.crispatus', 11: 'Lactobacillus.delbrueckii', 12: 'Lactobacillus.gasseri', 13: 'Lactobacillus.jehnsenii', 14: 'Lactobacillus.johnsonii', 15: 'Lactobacillus.paracasei', 16: 'Lactobacillus.plantarum', 17: 'Lactobacillus.reuteri', 18: 'Lactobacillus.rhamnosus', 19: 'Lactobacillus.salivarius', 20: 'Listeria.monocytogenes', 21: 'Micrococcus.spp', 22: 'Neisseria.gonorrhoeae', 23: 'Porfyromonas.gingivalis', 24: 'Propionibacterium.acnes', 25: 'Proteus', 26: 'Pseudomonas.aeruginosa', 27: 'Staphylococcus.aureus', 28: 'Staphylococcus.epidermidis', 29: 'Staphylococcus.saprophiticus', 30: 'Streptococcus.agalactiae', 31: 'Veionella'}


## Model definition and initialization

#### Freezing pre-trained parameters, finetunning the classifier to output 32 classes.

In [18]:
# Freeze pretrained model parameters to avoid backpropogating through them
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        print("Setting grad to false.")
        for param in model.parameters():
            param.requires_grad = False

    return model

def get_device():
    # Model and criterion to GPU
    if torch.cuda.is_available():
        return 'cuda'
    else:
        return 'cpu'

## Model (CNN + Tranformer)

In [19]:
def to_3d(x):
    return rearrange(x, 'b c h w -> b (h w) c')

def to_4d(x,h,w):
    return rearrange(x, 'b (h w) c -> b c h w',h=h,w=w)

class BiasFree_LayerNorm(nn.Module):
    def __init__(self, normalized_shape):
        super(BiasFree_LayerNorm, self).__init__()
        if isinstance(normalized_shape, numbers.Integral):
            normalized_shape = (normalized_shape,)
        normalized_shape = torch.Size(normalized_shape)

        assert len(normalized_shape) == 1

        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.normalized_shape = normalized_shape

    def forward(self, x):
        sigma = x.var(-1, keepdim=True, unbiased=False)
        return x / torch.sqrt(sigma+1e-5) * self.weight

class WithBias_LayerNorm(nn.Module):
    def __init__(self, normalized_shape):
        super(WithBias_LayerNorm, self).__init__()
        if isinstance(normalized_shape, numbers.Integral):
            normalized_shape = (normalized_shape,)
        normalized_shape = torch.Size(normalized_shape)

        assert len(normalized_shape) == 1

        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.normalized_shape = normalized_shape

    def forward(self, x):
        mu = x.mean(-1, keepdim=True)
        sigma = x.var(-1, keepdim=True, unbiased=False)
        return (x - mu) / torch.sqrt(sigma+1e-5) * self.weight + self.bias

class LayerNorm(nn.Module):
    def __init__(self, dim, LayerNorm_type):
        super(LayerNorm, self).__init__()
        if LayerNorm_type =='BiasFree':
            self.body = BiasFree_LayerNorm(dim)
        else:
            self.body = WithBias_LayerNorm(dim)

    def forward(self, x):
        h, w = x.shape[-2:]
        return to_4d(self.body(to_3d(x)), h, w)


class FeedForward(nn.Module):
    def __init__(self, dim, ffn_expansion_factor, bias):
        super(FeedForward, self).__init__()

        hidden_features = int(dim*ffn_expansion_factor)

        self.project_in = nn.Conv2d(dim, hidden_features*2, kernel_size=1, bias=bias)

        self.dwconv = nn.Conv2d(hidden_features*2, hidden_features*2, kernel_size=3, stride=1, padding=1, groups=hidden_features*2, bias=bias)

        self.project_out = nn.Conv2d(hidden_features, dim, kernel_size=1, bias=bias)

    def forward(self, x):
        x = self.project_in(x)
        x1, x2 = self.dwconv(x).chunk(2, dim=1)
        x = F.gelu(x1) * x2
        x = self.project_out(x)
        return x



class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, bias=False):
        super(TransformerBlock, self).__init__()

        # Initialize ScaledDotProduct attention
        self.num_heads = num_heads
        self.attention = ScaledDotProduct(
            dropout=0.1
        )

        # MLP (feedforward network) from Xformers
        # self.ffn = MLP(
        #     dim_model=embed_dim,
        #     dropout=0.1,
        #     activation="relu",
        #     hidden_layer_multiplier=4
        # )

        self.ffn = FeedForward(embed_dim, ffn_expansion_factor=2.66, bias=bias)

        # Layer normalization
        self.norm1 = LayerNorm(embed_dim, 'WithBias')
        self.norm2 = LayerNorm(embed_dim, 'WithBias')

        # Convolutional layers for QKV projections
        self.qkv = nn.Conv2d(embed_dim, embed_dim*3, kernel_size=1, bias=bias)
        self.qkv_dwconv = nn.Conv2d(embed_dim*3, embed_dim*3, kernel_size=3, stride=1, padding=1, groups=embed_dim*3, bias=bias)

    def forward(self, x):
        b, c, h, w = x.shape
        qkv = self.qkv_dwconv(self.qkv(x))
        q, k, v = qkv.chunk(3, dim=1)  # Split into query, key, value

        # Reshaping for multi-head attention
        q = rearrange(q, 'b (head c) h w -> b head c (h w)', head=self.num_heads)
        k = rearrange(k, 'b (head c) h w -> b head c (h w)', head=self.num_heads)
        v = rearrange(v, 'b (head c) h w -> b head c (h w)', head=self.num_heads)

        q = torch.nn.functional.normalize(q, dim=-1)
        k = torch.nn.functional.normalize(k, dim=-1)

        # Apply ScaledDotProduct attention
        attn_output = self.attention(q, k, v)

        # Reshape attention output to match input shape for residual connection
        # Ensure the flattened spatial dimensions match

        attn_output = rearrange(attn_output, 'b head c (h w) -> b (head c) h w', head=self.num_heads, h=h, w=w)

        # Add residual connection and layer normalization
        x = x + attn_output
        x = self.norm1(x)

        # x = x.flatten(2)  # Flatten spatial dimensions (height and width)
        # x = x.transpose(1, 2)  # Change shape to [batch_size, features, N] for MLP (N = height * width)

        # Apply FFN with residual connection
        ffn_output = self.ffn(x)
        x = x + ffn_output
        x = self.norm2(x)

        return x

class MobileNetTransformer(nn.Module):
    def __init__(self, num_classes=1000, num_heads=4, embed_dim=128):
        super(MobileNetTransformer, self).__init__()

        # Load MobileNetV2
        self.mobilenet = mobilenet_v2(pretrained=True)
        self.mobilenet.classifier = nn.Identity()  # Remove classifier layer

        # Freeze Mobilenet Weight
        for param in self.mobilenet.parameters():
            param.requires_grad = False

        # Define the output channels for each MobileNetV2 block based on your previous output
        block_output_channels = [
            32, 16, 24, 24, 32, 32, 32, 64, 64, 64, 64, 96, 96, 96, 160, 160, 160, 320, 1280
        ]

        # Create transformer blocks to match MobileNetV2 block structure
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(embed_dim=block_output_channels[i], num_heads=num_heads)
            for i in range(len(self.mobilenet.features))
        ])

        # Final classification layer
        self.fc = nn.Linear(block_output_channels[-1], num_classes)

    def forward(self, x):
        for i, mobilenet_block in enumerate(self.mobilenet.features):
            # Apply MobileNetV2 block
            x = mobilenet_block(x)

            # Get the number of output channels for this block
            # batch_size, channels, height, width = x.size()

            # Flatten to prepare for transformer input if necessary
            # x = x.view(batch_size, channels, height * width).transpose(1, 2)  # Shape [B, N, C]

            # Apply corresponding Transformer block
            x = self.transformer_blocks[i](x)

            # Reshape back to image format for the next MobileNetV2 block
            # x = x.transpose(1, 2).view(batch_size, channels, height, width)

        # Global average pooling for classification
        x = x.mean(dim=[2, 3])  # Shape [B, C]
        x = self.fc(x)  # Shape [B, num_classes]
        return x

# # Instantiate and test the model
# model = MobileNetTransformer(num_classes=32, num_heads=4, embed_dim=128)
# sample_input = torch.randn(1, 3, 224, 224)
# output = model(sample_input)
# print(output.shape)  # Should be [1, 32]





In [21]:
%cd /content

/content


In [22]:
def load_model():
    # Transfer Learning
    model = MobileNetTransformer(num_classes=32, num_heads=4, embed_dim=48)

    # Mode
    # model = set_parameter_requires_grad(model, feature_extract)

    # Fine tuning
    # Build custom classifier
    # model._fc = nn.Linear(in_features=1280,
    #                     out_features=32)
    return model

def create_optimizer(model):
    # Parameters to update
    params_to_update = model.parameters()

    if feature_extract:
        params_to_update = []
        for param in model.parameters():
            if param.requires_grad == True:
                params_to_update.append(param)

    else:
        n_params = 0
        for param in model.parameters():
            if param.requires_grad == True:
                n_params += 1


    # Loss function and gradient descent

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(params_to_update,
                          lr=0.001,
                          weight_decay=0.000004)

    return criterion.to(get_device()), model.to(get_device()), optimizer

## Training, validation and test functions

In [23]:
import os
save_dir = '/content/models'
os.makedirs(save_dir)

In [24]:
# Variables to store fold scores
train_acc = []
test_top1_acc = []
test_top5_acc = []
test_precision = []
test_recall = []
test_f1 = []
times = []

for fold, (train_idx, valid_idx) in enumerate(splits.split(total_set)):

    start_time = time.time()

    print('Fold : {}'.format(fold))

    # Train and val samplers
    train_sampler = SubsetRandomSampler(train_idx)
    print("Samples in training:", len(train_sampler))
    valid_sampler = SubsetRandomSampler(valid_idx)
    print("Samples in test:", len(valid_sampler))

    # Train and val loaders
    train_loader = torch.utils.data.DataLoader(
                      total_set, batch_size=train_batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(
                      total_set, batch_size=1, sampler=valid_sampler)

    device = get_device()

    criterion, model, optimizer = create_optimizer(load_model())

    # Print Model Summary
    print(summary(model, torch.zeros(1, 3, 224, 224).cuda()))

    # Training
    for epoch in range(h_epochs):

        model.train()
        running_loss = 0.0
        running_corrects = 0
        trunning_corrects = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += (preds == labels).sum()
            trunning_corrects += preds.size(0)


        epoch_loss = running_loss / trunning_corrects
        epoch_acc = (running_corrects.double()*100) / trunning_corrects
        train_acc.append(epoch_acc.item())

        print('\t\t Training: Epoch({}) - Loss: {:.4f}, Acc: {:.4f}'.format(epoch, epoch_loss, epoch_acc))

        # Validation

        model.eval()

        vrunning_loss = 0.0
        vrunning_corrects = 0
        num_samples = 0

        for data, labels in valid_loader:

            data = data.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.no_grad():
                outputs = model(data)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

            vrunning_loss += loss.item() * data.size(0)
            vrunning_corrects += (preds == labels).sum()
            num_samples += preds.size(0)

        vepoch_loss = vrunning_loss/num_samples
        vepoch_acc = (vrunning_corrects.double() * 100)/num_samples

        print('\t\t Validation({}) - Loss: {:.4f}, Acc: {:.4f}'.format(epoch, vepoch_loss, vepoch_acc))

        # Save Models
        model_save_path = os.path.join(save_dir, f"model_fold_{fold}_epoch_{epoch}.pth")
        torch.save(model.state_dict(), model_save_path)
        print(f"Model for Fold {fold}, Epoch {epoch} saved at {model_save_path}")

    # Calculating and appending scores to this fold
    model.class_to_idx = total_set.class_to_idx
    scores = get_scores(model, valid_loader)

    test_top1_acc.append(scores[0])
    test_top5_acc.append(scores[1])
    test_precision.append(scores[2])
    test_recall.append(scores[3])
    test_f1.append(scores[4])

    time_fold = time.time() - start_time
    times.append(time_fold)
    print("Total time per fold: %s seconds." %(time_fold))

Fold : 0
Samples in training: 602
Samples in test: 67


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 180MB/s]
/usr/local/lib/python3.10/dist-packages/xformers/components/attention/base.py:40: FutureWarning: ScaledDotProduct() is deprecated and is not maintained anymore. It might be rem

-----------------------------------------------------------------------------
          Layer (type)          Output Shape         Param #     Tr. Param #
    TransformerBlock-1     [1, 32, 112, 112]          13,754          13,754
    TransformerBlock-2     [1, 16, 112, 112]           4,036           4,036
    TransformerBlock-3       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-4       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-5       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-6       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-7       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-8       [1, 64, 14, 14]          49,972          49,972
    TransformerBlock-9       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-10       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-11       [1, 64, 14, 14]          49,972          49,97

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/xformers/components/attention/base.py:40: FutureWarning: ScaledDotProduct() is deprecated and is not maintained anymore. It might be removed in a future version of xFormers
  deprecated_function(self)


-----------------------------------------------------------------------------
          Layer (type)          Output Shape         Param #     Tr. Param #
    TransformerBlock-1     [1, 32, 112, 112]          13,754          13,754
    TransformerBlock-2     [1, 16, 112, 112]           4,036           4,036
    TransformerBlock-3       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-4       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-5       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-6       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-7       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-8       [1, 64, 14, 14]          49,972          49,972
    TransformerBlock-9       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-10       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-11       [1, 64, 14, 14]          49,972          49,97

KeyboardInterrupt: 

In [ ]:
print("Train accuracy average: ", np.mean(train_acc) / 100)
print("Top-1 test accuracy average: ", np.mean(test_top1_acc))
print("Top-5 test accuracy average: ", np.mean(test_top5_acc))
print("Weighted Precision test accuracy average: ", np.mean(test_precision))
print("Weighted Recall test accuracy average: ", np.mean(test_recall))
print("Weighted F1 test accuracy average: ", np.mean(test_f1))
print("Average time per fold (seconds):", np.mean(times))

Train accuracy average:  0.9274636295949993
Top-1 test accuracy average:  0.9317561807331629
Top-5 test accuracy average:  0.9942028985507246
Weighted Precision test accuracy average:  0.9483380695273278
Weighted Recall test accuracy average:  0.9317561807331629
Weighted F1 test accuracy average:  0.9319668660173775
Average time per fold (seconds): 654.8511736869812


In [32]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.0+cu121 requires torch==2.5.0, but you have torch 2.5.1 which is incompatible.
torchvisio

In [41]:
import gradio as gr
import torch
from PIL import Image
import io

# Load the best model
def load_best_model(path):
    model = MobileNetTransformer(num_classes=32, num_heads=4, embed_dim=48)
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

# Convert TIFF bytes to JPG for preview, then classify using the original TIFF data
def handle_upload(tiff_file):
    # Convert bytes to an image for both preview and processing
    tiff_bytes = io.BytesIO(tiff_file)  # Wrap bytes in a BytesIO object for PIL

    # Convert TIFF to JPG for preview
    with Image.open(tiff_bytes) as img:
        img_jpg = img.convert("RGB")
        jpg_buffer = io.BytesIO()
        img_jpg.save(jpg_buffer, format="JPEG")
        jpg_buffer.seek(0)
        preview_image = Image.open(jpg_buffer)

    # Rewind the BytesIO object for original TIFF classification
    tiff_bytes.seek(0)
    with Image.open(tiff_bytes) as original_img:
        image = training_transforms(original_img).unsqueeze(0)
        with torch.no_grad():
            outputs = best_model(image.to(get_device()))
            _, predicted = torch.max(outputs, 1)
    return preview_image, train_labels[predicted.item()]  # Return JPG preview and label

# Load the model
best_model_path = "models/model_fold_0_epoch_14.pth"
best_model = load_best_model(best_model_path).to(get_device())

# Gradio interface setup
with gr.Blocks() as gr_interface:
    with gr.Row():
        with gr.Column():
            file_input = gr.File(type="binary", label="Upload TIFF File")
            image_preview = gr.Image(type="pil", label="Preview (JPG)", interactive=False)
        label_output = gr.Textbox(label="Prediction")

    file_input.change(handle_upload, inputs=file_input, outputs=[image_preview, label_output])

gr_interface.launch(debug=True)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/xformers/components/attention/base.py:40: FutureWarning: ScaledDotProduct() is deprecated and is not maintained anymore. It might be removed in a future version of xFormers
  deprecated_function(self)
<ipython-input-41-9949ff7b950a>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default valu

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://54e6813a3fb6a6c743.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://54e6813a3fb6a6c743.gradio.live


## Full Train Log

In [ ]:
'''
Fold : 0
Samples in training: 620
Samples in test: 69
-----------------------------------------------------------------------------
          Layer (type)          Output Shape         Param #     Tr. Param #
=============================================================================
    TransformerBlock-1     [1, 32, 112, 112]          13,754          13,754
    TransformerBlock-2     [1, 16, 112, 112]           4,036           4,036
    TransformerBlock-3       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-4       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-5       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-6       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-7       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-8       [1, 64, 14, 14]          49,972          49,972
    TransformerBlock-9       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-10       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-11       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-12       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-13       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-14       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-15        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-16        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-17        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-18        [1, 320, 7, 7]       1,149,398       1,149,398
   TransformerBlock-19       [1, 1280, 7, 7]      18,087,512      18,087,512
             Linear-20               [1, 32]          40,992          40,992
=============================================================================
Total params: 20,759,318
Trainable params: 20,759,318
Non-trainable params: 0
-----------------------------------------------------------------------------
		 Training: Epoch(0) - Loss: 1.6021, Acc: 55.9677
		 Validation(0) - Loss: 1.7764, Acc: 59.4203
		 Training: Epoch(1) - Loss: 0.3163, Acc: 89.8387
		 Validation(1) - Loss: 0.8265, Acc: 81.1594
		 Training: Epoch(2) - Loss: 0.3694, Acc: 88.8710
		 Validation(2) - Loss: 0.3940, Acc: 86.9565
		 Training: Epoch(3) - Loss: 0.1728, Acc: 94.8387
		 Validation(3) - Loss: 0.3483, Acc: 89.8551
		 Training: Epoch(4) - Loss: 0.1557, Acc: 94.8387
		 Validation(4) - Loss: 0.5214, Acc: 88.4058
		 Training: Epoch(5) - Loss: 0.1412, Acc: 94.8387
		 Validation(5) - Loss: 0.6390, Acc: 85.5072
		 Training: Epoch(6) - Loss: 0.1792, Acc: 95.9677
		 Validation(6) - Loss: 0.7840, Acc: 85.5072
		 Training: Epoch(7) - Loss: 0.1138, Acc: 96.7742
		 Validation(7) - Loss: 0.4303, Acc: 88.4058
		 Training: Epoch(8) - Loss: 0.0480, Acc: 98.5484
		 Validation(8) - Loss: 0.4316, Acc: 88.4058
		 Training: Epoch(9) - Loss: 0.0622, Acc: 97.9032
		 Validation(9) - Loss: 0.4203, Acc: 92.7536
		 Training: Epoch(10) - Loss: 0.0964, Acc: 97.5806
		 Validation(10) - Loss: 0.7494, Acc: 86.9565
		 Training: Epoch(11) - Loss: 0.0551, Acc: 98.3871
		 Validation(11) - Loss: 0.4108, Acc: 88.4058
		 Training: Epoch(12) - Loss: 0.0736, Acc: 97.2581
		 Validation(12) - Loss: 0.2768, Acc: 94.2029
		 Training: Epoch(13) - Loss: 0.0138, Acc: 99.5161
		 Validation(13) - Loss: 0.3203, Acc: 89.8551
		 Training: Epoch(14) - Loss: 0.0096, Acc: 99.6774
		 Validation(14) - Loss: 0.2821, Acc: 91.3043
Finished.
Total time per fold: 669.8467693328857 seconds.
Fold : 1
Samples in training: 620
Samples in test: 69
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/xformers/components/attention/base.py:40: FutureWarning: ScaledDotProduct() is deprecated and is not maintained anymore. It might be removed in a future version of xFormers
  deprecated_function(self)
-----------------------------------------------------------------------------
          Layer (type)          Output Shape         Param #     Tr. Param #
=============================================================================
    TransformerBlock-1     [1, 32, 112, 112]          13,754          13,754
    TransformerBlock-2     [1, 16, 112, 112]           4,036           4,036
    TransformerBlock-3       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-4       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-5       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-6       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-7       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-8       [1, 64, 14, 14]          49,972          49,972
    TransformerBlock-9       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-10       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-11       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-12       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-13       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-14       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-15        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-16        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-17        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-18        [1, 320, 7, 7]       1,149,398       1,149,398
   TransformerBlock-19       [1, 1280, 7, 7]      18,087,512      18,087,512
             Linear-20               [1, 32]          40,992          40,992
=============================================================================
Total params: 20,759,318
Trainable params: 20,759,318
Non-trainable params: 0
-----------------------------------------------------------------------------
		 Training: Epoch(0) - Loss: 1.6578, Acc: 55.9677
		 Validation(0) - Loss: 1.7942, Acc: 47.8261
		 Training: Epoch(1) - Loss: 0.3869, Acc: 86.9355
		 Validation(1) - Loss: 0.4094, Acc: 89.8551
		 Training: Epoch(2) - Loss: 0.3034, Acc: 91.1290
		 Validation(2) - Loss: 0.4581, Acc: 89.8551
		 Training: Epoch(3) - Loss: 0.3752, Acc: 89.6774
		 Validation(3) - Loss: 0.4943, Acc: 85.5072
		 Training: Epoch(4) - Loss: 0.1716, Acc: 95.3226
		 Validation(4) - Loss: 0.2245, Acc: 91.3043
		 Training: Epoch(5) - Loss: 0.1660, Acc: 95.3226
		 Validation(5) - Loss: 0.7475, Acc: 85.5072
		 Training: Epoch(6) - Loss: 0.1300, Acc: 96.4516
		 Validation(6) - Loss: 0.5309, Acc: 89.8551
		 Training: Epoch(7) - Loss: 0.0744, Acc: 97.5806
		 Validation(7) - Loss: 0.4378, Acc: 91.3043
		 Training: Epoch(8) - Loss: 0.0769, Acc: 98.2258
		 Validation(8) - Loss: 0.3282, Acc: 92.7536
		 Training: Epoch(9) - Loss: 0.1131, Acc: 96.9355
		 Validation(9) - Loss: 0.7955, Acc: 85.5072
		 Training: Epoch(10) - Loss: 0.1871, Acc: 95.9677
		 Validation(10) - Loss: 0.5083, Acc: 91.3043
		 Training: Epoch(11) - Loss: 0.0844, Acc: 97.9032
		 Validation(11) - Loss: 0.5079, Acc: 95.6522
		 Training: Epoch(12) - Loss: 0.0613, Acc: 97.9032
		 Validation(12) - Loss: 0.2177, Acc: 91.3043
		 Training: Epoch(13) - Loss: 0.0154, Acc: 99.5161
		 Validation(13) - Loss: 0.2138, Acc: 91.3043
		 Training: Epoch(14) - Loss: 0.0223, Acc: 99.3548
		 Validation(14) - Loss: 0.4489, Acc: 92.7536
Finished.
Total time per fold: 656.4339890480042 seconds.
Fold : 2
Samples in training: 620
Samples in test: 69
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/xformers/components/attention/base.py:40: FutureWarning: ScaledDotProduct() is deprecated and is not maintained anymore. It might be removed in a future version of xFormers
  deprecated_function(self)
-----------------------------------------------------------------------------
          Layer (type)          Output Shape         Param #     Tr. Param #
=============================================================================
    TransformerBlock-1     [1, 32, 112, 112]          13,754          13,754
    TransformerBlock-2     [1, 16, 112, 112]           4,036           4,036
    TransformerBlock-3       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-4       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-5       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-6       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-7       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-8       [1, 64, 14, 14]          49,972          49,972
    TransformerBlock-9       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-10       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-11       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-12       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-13       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-14       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-15        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-16        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-17        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-18        [1, 320, 7, 7]       1,149,398       1,149,398
   TransformerBlock-19       [1, 1280, 7, 7]      18,087,512      18,087,512
             Linear-20               [1, 32]          40,992          40,992
=============================================================================
Total params: 20,759,318
Trainable params: 20,759,318
Non-trainable params: 0
-----------------------------------------------------------------------------
		 Training: Epoch(0) - Loss: 1.6379, Acc: 55.4839
		 Validation(0) - Loss: 1.4442, Acc: 56.5217
		 Training: Epoch(1) - Loss: 0.5617, Acc: 83.3871
		 Validation(1) - Loss: 0.8004, Acc: 82.6087
		 Training: Epoch(2) - Loss: 0.4036, Acc: 87.0968
		 Validation(2) - Loss: 0.3178, Acc: 91.3043
		 Training: Epoch(3) - Loss: 0.2980, Acc: 90.0000
		 Validation(3) - Loss: 0.2142, Acc: 92.7536
		 Training: Epoch(4) - Loss: 0.1591, Acc: 95.6452
		 Validation(4) - Loss: 0.2288, Acc: 91.3043
		 Training: Epoch(5) - Loss: 0.1016, Acc: 95.9677
		 Validation(5) - Loss: 0.2956, Acc: 91.3043
		 Training: Epoch(6) - Loss: 0.1457, Acc: 96.2903
		 Validation(6) - Loss: 0.1694, Acc: 92.7536
		 Training: Epoch(7) - Loss: 0.0832, Acc: 97.5806
		 Validation(7) - Loss: 0.3022, Acc: 95.6522
		 Training: Epoch(8) - Loss: 0.0993, Acc: 97.4194
		 Validation(8) - Loss: 0.1473, Acc: 95.6522
		 Training: Epoch(9) - Loss: 0.1730, Acc: 96.6129
		 Validation(9) - Loss: 0.5352, Acc: 89.8551
		 Training: Epoch(10) - Loss: 0.1130, Acc: 96.6129
		 Validation(10) - Loss: 0.5970, Acc: 89.8551
		 Training: Epoch(11) - Loss: 0.1226, Acc: 97.4194
		 Validation(11) - Loss: 0.1235, Acc: 95.6522
		 Training: Epoch(12) - Loss: 0.0803, Acc: 97.5806
		 Validation(12) - Loss: 0.2146, Acc: 92.7536
		 Training: Epoch(13) - Loss: 0.0715, Acc: 98.3871
		 Validation(13) - Loss: 0.2671, Acc: 95.6522
		 Training: Epoch(14) - Loss: 0.0935, Acc: 98.0645
		 Validation(14) - Loss: 0.2244, Acc: 92.7536
Finished.
Total time per fold: 656.0651466846466 seconds.
Fold : 3
Samples in training: 620
Samples in test: 69
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/xformers/components/attention/base.py:40: FutureWarning: ScaledDotProduct() is deprecated and is not maintained anymore. It might be removed in a future version of xFormers
  deprecated_function(self)
-----------------------------------------------------------------------------
          Layer (type)          Output Shape         Param #     Tr. Param #
=============================================================================
    TransformerBlock-1     [1, 32, 112, 112]          13,754          13,754
    TransformerBlock-2     [1, 16, 112, 112]           4,036           4,036
    TransformerBlock-3       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-4       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-5       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-6       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-7       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-8       [1, 64, 14, 14]          49,972          49,972
    TransformerBlock-9       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-10       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-11       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-12       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-13       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-14       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-15        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-16        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-17        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-18        [1, 320, 7, 7]       1,149,398       1,149,398
   TransformerBlock-19       [1, 1280, 7, 7]      18,087,512      18,087,512
             Linear-20               [1, 32]          40,992          40,992
=============================================================================
Total params: 20,759,318
Trainable params: 20,759,318
Non-trainable params: 0
-----------------------------------------------------------------------------
		 Training: Epoch(0) - Loss: 1.5997, Acc: 57.0968
		 Validation(0) - Loss: 1.7421, Acc: 53.6232
		 Training: Epoch(1) - Loss: 0.4651, Acc: 85.1613
		 Validation(1) - Loss: 0.5389, Acc: 85.5072
		 Training: Epoch(2) - Loss: 0.2826, Acc: 92.0968
		 Validation(2) - Loss: 0.3839, Acc: 89.8551
		 Training: Epoch(3) - Loss: 0.2425, Acc: 91.9355
		 Validation(3) - Loss: 0.2250, Acc: 92.7536
		 Training: Epoch(4) - Loss: 0.2013, Acc: 94.8387
		 Validation(4) - Loss: 0.3153, Acc: 88.4058
		 Training: Epoch(5) - Loss: 0.1751, Acc: 93.7097
		 Validation(5) - Loss: 0.4091, Acc: 86.9565
		 Training: Epoch(6) - Loss: 0.0958, Acc: 96.9355
		 Validation(6) - Loss: 0.3122, Acc: 91.3043
		 Training: Epoch(7) - Loss: 0.1601, Acc: 95.1613
		 Validation(7) - Loss: 0.2990, Acc: 92.7536
		 Training: Epoch(8) - Loss: 0.0717, Acc: 97.4194
		 Validation(8) - Loss: 0.3687, Acc: 92.7536
		 Training: Epoch(9) - Loss: 0.0972, Acc: 97.9032
		 Validation(9) - Loss: 0.1897, Acc: 94.2029
		 Training: Epoch(10) - Loss: 0.0127, Acc: 99.6774
		 Validation(10) - Loss: 0.1812, Acc: 94.2029
		 Training: Epoch(11) - Loss: 0.0359, Acc: 99.0323
		 Validation(11) - Loss: 0.2605, Acc: 91.3043
		 Training: Epoch(12) - Loss: 0.0118, Acc: 99.5161
		 Validation(12) - Loss: 0.3594, Acc: 91.3043
		 Training: Epoch(13) - Loss: 0.0455, Acc: 98.8710
		 Validation(13) - Loss: 0.1670, Acc: 94.2029
		 Training: Epoch(14) - Loss: 0.0286, Acc: 98.8710
		 Validation(14) - Loss: 0.5240, Acc: 92.7536
Finished.
Total time per fold: 646.6555397510529 seconds.
Fold : 4
Samples in training: 620
Samples in test: 69
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/xformers/components/attention/base.py:40: FutureWarning: ScaledDotProduct() is deprecated and is not maintained anymore. It might be removed in a future version of xFormers
  deprecated_function(self)
-----------------------------------------------------------------------------
          Layer (type)          Output Shape         Param #     Tr. Param #
=============================================================================
    TransformerBlock-1     [1, 32, 112, 112]          13,754          13,754
    TransformerBlock-2     [1, 16, 112, 112]           4,036           4,036
    TransformerBlock-3       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-4       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-5       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-6       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-7       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-8       [1, 64, 14, 14]          49,972          49,972
    TransformerBlock-9       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-10       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-11       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-12       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-13       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-14       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-15        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-16        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-17        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-18        [1, 320, 7, 7]       1,149,398       1,149,398
   TransformerBlock-19       [1, 1280, 7, 7]      18,087,512      18,087,512
             Linear-20               [1, 32]          40,992          40,992
=============================================================================
Total params: 20,759,318
Trainable params: 20,759,318
Non-trainable params: 0
-----------------------------------------------------------------------------
		 Training: Epoch(0) - Loss: 1.7609, Acc: 55.0000
		 Validation(0) - Loss: 1.2189, Acc: 63.7681
		 Training: Epoch(1) - Loss: 0.5894, Acc: 83.2258
		 Validation(1) - Loss: 0.3059, Acc: 92.7536
		 Training: Epoch(2) - Loss: 0.4547, Acc: 86.9355
		 Validation(2) - Loss: 0.4786, Acc: 84.0580
		 Training: Epoch(3) - Loss: 0.2852, Acc: 90.0000
		 Validation(3) - Loss: 0.2872, Acc: 88.4058
		 Training: Epoch(4) - Loss: 0.1536, Acc: 95.1613
		 Validation(4) - Loss: 0.2094, Acc: 91.3043
		 Training: Epoch(5) - Loss: 0.1197, Acc: 96.2903
		 Validation(5) - Loss: 0.1281, Acc: 94.2029
		 Training: Epoch(6) - Loss: 0.1094, Acc: 97.2581
		 Validation(6) - Loss: 0.3312, Acc: 92.7536
		 Training: Epoch(7) - Loss: 0.0513, Acc: 98.5484
		 Validation(7) - Loss: 0.4139, Acc: 89.8551
		 Training: Epoch(8) - Loss: 0.1145, Acc: 96.7742
		 Validation(8) - Loss: 0.1838, Acc: 94.2029
		 Training: Epoch(9) - Loss: 0.0454, Acc: 98.2258
		 Validation(9) - Loss: 0.2753, Acc: 91.3043
		 Training: Epoch(10) - Loss: 0.0561, Acc: 97.7419
		 Validation(10) - Loss: 0.3317, Acc: 89.8551
		 Training: Epoch(11) - Loss: 0.1005, Acc: 97.4194
		 Validation(11) - Loss: 0.3123, Acc: 89.8551
		 Training: Epoch(12) - Loss: 0.0334, Acc: 98.5484
		 Validation(12) - Loss: 0.2477, Acc: 91.3043
		 Training: Epoch(13) - Loss: 0.0574, Acc: 98.3871
		 Validation(13) - Loss: 0.1434, Acc: 92.7536
		 Training: Epoch(14) - Loss: 0.0538, Acc: 99.0323
		 Validation(14) - Loss: 0.2347, Acc: 91.3043
Finished.
Total time per fold: 649.2717716693878 seconds.
Fold : 5
Samples in training: 620
Samples in test: 69
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/xformers/components/attention/base.py:40: FutureWarning: ScaledDotProduct() is deprecated and is not maintained anymore. It might be removed in a future version of xFormers
  deprecated_function(self)
-----------------------------------------------------------------------------
          Layer (type)          Output Shape         Param #     Tr. Param #
=============================================================================
    TransformerBlock-1     [1, 32, 112, 112]          13,754          13,754
    TransformerBlock-2     [1, 16, 112, 112]           4,036           4,036
    TransformerBlock-3       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-4       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-5       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-6       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-7       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-8       [1, 64, 14, 14]          49,972          49,972
    TransformerBlock-9       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-10       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-11       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-12       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-13       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-14       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-15        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-16        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-17        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-18        [1, 320, 7, 7]       1,149,398       1,149,398
   TransformerBlock-19       [1, 1280, 7, 7]      18,087,512      18,087,512
             Linear-20               [1, 32]          40,992          40,992
=============================================================================
Total params: 20,759,318
Trainable params: 20,759,318
Non-trainable params: 0
-----------------------------------------------------------------------------
		 Training: Epoch(0) - Loss: 1.7177, Acc: 55.4839
		 Validation(0) - Loss: 2.0823, Acc: 47.8261
		 Training: Epoch(1) - Loss: 0.5853, Acc: 80.9677
		 Validation(1) - Loss: 0.5014, Acc: 86.9565
		 Training: Epoch(2) - Loss: 0.3458, Acc: 89.3548
		 Validation(2) - Loss: 0.4835, Acc: 85.5072
		 Training: Epoch(3) - Loss: 0.2926, Acc: 91.2903
		 Validation(3) - Loss: 0.5963, Acc: 86.9565
		 Training: Epoch(4) - Loss: 0.1988, Acc: 94.3548
		 Validation(4) - Loss: 0.4931, Acc: 89.8551
		 Training: Epoch(5) - Loss: 0.1263, Acc: 96.1290
		 Validation(5) - Loss: 0.2822, Acc: 92.7536
		 Training: Epoch(6) - Loss: 0.1513, Acc: 95.8065
		 Validation(6) - Loss: 0.5524, Acc: 86.9565
		 Training: Epoch(7) - Loss: 0.1180, Acc: 96.7742
		 Validation(7) - Loss: 0.2433, Acc: 91.3043
		 Training: Epoch(8) - Loss: 0.0945, Acc: 97.4194
		 Validation(8) - Loss: 0.2529, Acc: 94.2029
		 Training: Epoch(9) - Loss: 0.1632, Acc: 96.4516
		 Validation(9) - Loss: 0.4000, Acc: 84.0580
		 Training: Epoch(10) - Loss: 0.0628, Acc: 98.3871
		 Validation(10) - Loss: 0.2364, Acc: 92.7536
		 Training: Epoch(11) - Loss: 0.0682, Acc: 98.7097
		 Validation(11) - Loss: 0.2897, Acc: 89.8551
		 Training: Epoch(12) - Loss: 0.0376, Acc: 99.3548
		 Validation(12) - Loss: 0.1112, Acc: 95.6522
		 Training: Epoch(13) - Loss: 0.0198, Acc: 99.5161
		 Validation(13) - Loss: 0.1047, Acc: 95.6522
		 Training: Epoch(14) - Loss: 0.0580, Acc: 99.0323
		 Validation(14) - Loss: 0.0700, Acc: 98.5507
Finished.
Total time per fold: 656.2941517829895 seconds.
Fold : 6
Samples in training: 620
Samples in test: 69
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/xformers/components/attention/base.py:40: FutureWarning: ScaledDotProduct() is deprecated and is not maintained anymore. It might be removed in a future version of xFormers
  deprecated_function(self)
-----------------------------------------------------------------------------
          Layer (type)          Output Shape         Param #     Tr. Param #
=============================================================================
    TransformerBlock-1     [1, 32, 112, 112]          13,754          13,754
    TransformerBlock-2     [1, 16, 112, 112]           4,036           4,036
    TransformerBlock-3       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-4       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-5       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-6       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-7       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-8       [1, 64, 14, 14]          49,972          49,972
    TransformerBlock-9       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-10       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-11       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-12       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-13       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-14       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-15        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-16        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-17        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-18        [1, 320, 7, 7]       1,149,398       1,149,398
   TransformerBlock-19       [1, 1280, 7, 7]      18,087,512      18,087,512
             Linear-20               [1, 32]          40,992          40,992
=============================================================================
Total params: 20,759,318
Trainable params: 20,759,318
Non-trainable params: 0
-----------------------------------------------------------------------------
		 Training: Epoch(0) - Loss: 1.6610, Acc: 55.4839
		 Validation(0) - Loss: 1.5797, Acc: 56.5217
		 Training: Epoch(1) - Loss: 0.4162, Acc: 86.6129
		 Validation(1) - Loss: 0.4281, Acc: 86.9565
		 Training: Epoch(2) - Loss: 0.3656, Acc: 88.2258
		 Validation(2) - Loss: 0.4516, Acc: 89.8551
		 Training: Epoch(3) - Loss: 0.2919, Acc: 91.1290
		 Validation(3) - Loss: 0.3251, Acc: 91.3043
		 Training: Epoch(4) - Loss: 0.2362, Acc: 94.1935
		 Validation(4) - Loss: 0.5021, Acc: 88.4058
		 Training: Epoch(5) - Loss: 0.0937, Acc: 97.2581
		 Validation(5) - Loss: 0.3423, Acc: 89.8551
		 Training: Epoch(6) - Loss: 0.1173, Acc: 97.4194
		 Validation(6) - Loss: 0.3652, Acc: 89.8551
		 Training: Epoch(7) - Loss: 0.1183, Acc: 96.1290
		 Validation(7) - Loss: 0.2434, Acc: 92.7536
		 Training: Epoch(8) - Loss: 0.1273, Acc: 96.7742
		 Validation(8) - Loss: 0.3416, Acc: 89.8551
		 Training: Epoch(9) - Loss: 0.1570, Acc: 95.6452
		 Validation(9) - Loss: 0.2990, Acc: 92.7536
		 Training: Epoch(10) - Loss: 0.1088, Acc: 97.5806
		 Validation(10) - Loss: 0.5526, Acc: 86.9565
		 Training: Epoch(11) - Loss: 0.0804, Acc: 97.2581
		 Validation(11) - Loss: 0.4313, Acc: 88.4058
		 Training: Epoch(12) - Loss: 0.0400, Acc: 99.0323
		 Validation(12) - Loss: 0.4225, Acc: 89.8551
		 Training: Epoch(13) - Loss: 0.0464, Acc: 98.2258
		 Validation(13) - Loss: 0.3482, Acc: 94.2029
		 Training: Epoch(14) - Loss: 0.0464, Acc: 98.2258
		 Validation(14) - Loss: 0.3336, Acc: 92.7536
Finished.
Total time per fold: 655.8265824317932 seconds.
Fold : 7
Samples in training: 620
Samples in test: 69
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/xformers/components/attention/base.py:40: FutureWarning: ScaledDotProduct() is deprecated and is not maintained anymore. It might be removed in a future version of xFormers
  deprecated_function(self)
-----------------------------------------------------------------------------
          Layer (type)          Output Shape         Param #     Tr. Param #
=============================================================================
    TransformerBlock-1     [1, 32, 112, 112]          13,754          13,754
    TransformerBlock-2     [1, 16, 112, 112]           4,036           4,036
    TransformerBlock-3       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-4       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-5       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-6       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-7       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-8       [1, 64, 14, 14]          49,972          49,972
    TransformerBlock-9       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-10       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-11       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-12       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-13       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-14       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-15        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-16        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-17        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-18        [1, 320, 7, 7]       1,149,398       1,149,398
   TransformerBlock-19       [1, 1280, 7, 7]      18,087,512      18,087,512
             Linear-20               [1, 32]          40,992          40,992
=============================================================================
Total params: 20,759,318
Trainable params: 20,759,318
Non-trainable params: 0
-----------------------------------------------------------------------------
		 Training: Epoch(0) - Loss: 1.6276, Acc: 57.4194
		 Validation(0) - Loss: 1.1767, Acc: 63.7681
		 Training: Epoch(1) - Loss: 0.4749, Acc: 84.6774
		 Validation(1) - Loss: 0.5479, Acc: 81.1594
		 Training: Epoch(2) - Loss: 0.4465, Acc: 86.7742
		 Validation(2) - Loss: 0.3537, Acc: 88.4058
		 Training: Epoch(3) - Loss: 0.3416, Acc: 90.8065
		 Validation(3) - Loss: 0.5831, Acc: 86.9565
		 Training: Epoch(4) - Loss: 0.2634, Acc: 92.9032
		 Validation(4) - Loss: 0.3241, Acc: 89.8551
		 Training: Epoch(5) - Loss: 0.1770, Acc: 95.9677
		 Validation(5) - Loss: 0.1593, Acc: 97.1014
		 Training: Epoch(6) - Loss: 0.0888, Acc: 97.5806
		 Validation(6) - Loss: 0.4496, Acc: 92.7536
		 Training: Epoch(7) - Loss: 0.0597, Acc: 97.9032
		 Validation(7) - Loss: 0.2113, Acc: 94.2029
		 Training: Epoch(8) - Loss: 0.0846, Acc: 97.9032
		 Validation(8) - Loss: 0.3210, Acc: 92.7536
		 Training: Epoch(9) - Loss: 0.0699, Acc: 98.2258
		 Validation(9) - Loss: 0.2608, Acc: 95.6522
		 Training: Epoch(10) - Loss: 0.2384, Acc: 96.6129
		 Validation(10) - Loss: 0.5815, Acc: 88.4058
		 Training: Epoch(11) - Loss: 0.1032, Acc: 97.0968
		 Validation(11) - Loss: 0.2060, Acc: 97.1014
		 Training: Epoch(12) - Loss: 0.0616, Acc: 98.3871
		 Validation(12) - Loss: 0.1328, Acc: 97.1014
		 Training: Epoch(13) - Loss: 0.0409, Acc: 98.7097
		 Validation(13) - Loss: 0.0995, Acc: 95.6522
		 Training: Epoch(14) - Loss: 0.1104, Acc: 97.7419
		 Validation(14) - Loss: 0.2306, Acc: 95.6522
Finished.
Total time per fold: 649.9659056663513 seconds.
Fold : 8
Samples in training: 620
Samples in test: 69
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/xformers/components/attention/base.py:40: FutureWarning: ScaledDotProduct() is deprecated and is not maintained anymore. It might be removed in a future version of xFormers
  deprecated_function(self)
-----------------------------------------------------------------------------
          Layer (type)          Output Shape         Param #     Tr. Param #
=============================================================================
    TransformerBlock-1     [1, 32, 112, 112]          13,754          13,754
    TransformerBlock-2     [1, 16, 112, 112]           4,036           4,036
    TransformerBlock-3       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-4       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-5       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-6       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-7       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-8       [1, 64, 14, 14]          49,972          49,972
    TransformerBlock-9       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-10       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-11       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-12       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-13       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-14       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-15        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-16        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-17        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-18        [1, 320, 7, 7]       1,149,398       1,149,398
   TransformerBlock-19       [1, 1280, 7, 7]      18,087,512      18,087,512
             Linear-20               [1, 32]          40,992          40,992
=============================================================================
Total params: 20,759,318
Trainable params: 20,759,318
Non-trainable params: 0
-----------------------------------------------------------------------------
		 Training: Epoch(0) - Loss: 1.5873, Acc: 57.9032
		 Validation(0) - Loss: 1.9093, Acc: 52.1739
		 Training: Epoch(1) - Loss: 0.4703, Acc: 83.3871
		 Validation(1) - Loss: 0.5094, Acc: 86.9565
		 Training: Epoch(2) - Loss: 0.2275, Acc: 92.5806
		 Validation(2) - Loss: 0.2865, Acc: 91.3043
		 Training: Epoch(3) - Loss: 0.2198, Acc: 93.5484
		 Validation(3) - Loss: 0.1822, Acc: 94.2029
		 Training: Epoch(4) - Loss: 0.2174, Acc: 93.5484
		 Validation(4) - Loss: 0.6397, Acc: 89.8551
		 Training: Epoch(5) - Loss: 0.1751, Acc: 95.1613
		 Validation(5) - Loss: 0.2521, Acc: 94.2029
		 Training: Epoch(6) - Loss: 0.1970, Acc: 94.3548
		 Validation(6) - Loss: 0.5809, Acc: 89.8551
		 Training: Epoch(7) - Loss: 0.2503, Acc: 93.8710
		 Validation(7) - Loss: 0.4294, Acc: 88.4058
		 Training: Epoch(8) - Loss: 0.1182, Acc: 96.2903
		 Validation(8) - Loss: 0.3031, Acc: 91.3043
		 Training: Epoch(9) - Loss: 0.0937, Acc: 97.7419
		 Validation(9) - Loss: 0.3464, Acc: 94.2029
		 Training: Epoch(10) - Loss: 0.0338, Acc: 99.1935
		 Validation(10) - Loss: 0.3562, Acc: 92.7536
		 Training: Epoch(11) - Loss: 0.0394, Acc: 99.1935
		 Validation(11) - Loss: 0.3197, Acc: 95.6522
		 Training: Epoch(12) - Loss: 0.0098, Acc: 99.6774
		 Validation(12) - Loss: 0.2450, Acc: 94.2029
		 Training: Epoch(13) - Loss: 0.0732, Acc: 98.2258
		 Validation(13) - Loss: 0.5229, Acc: 88.4058
		 Training: Epoch(14) - Loss: 0.1150, Acc: 96.7742
		 Validation(14) - Loss: 0.2533, Acc: 92.7536
Finished.
Total time per fold: 653.4133305549622 seconds.
Fold : 9
Samples in training: 621
Samples in test: 68
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/xformers/components/attention/base.py:40: FutureWarning: ScaledDotProduct() is deprecated and is not maintained anymore. It might be removed in a future version of xFormers
  deprecated_function(self)
-----------------------------------------------------------------------------
          Layer (type)          Output Shape         Param #     Tr. Param #
=============================================================================
    TransformerBlock-1     [1, 32, 112, 112]          13,754          13,754
    TransformerBlock-2     [1, 16, 112, 112]           4,036           4,036
    TransformerBlock-3       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-4       [1, 24, 56, 56]           8,142           8,142
    TransformerBlock-5       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-6       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-7       [1, 32, 28, 28]          13,754          13,754
    TransformerBlock-8       [1, 64, 14, 14]          49,972          49,972
    TransformerBlock-9       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-10       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-11       [1, 64, 14, 14]          49,972          49,972
   TransformerBlock-12       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-13       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-14       [1, 96, 14, 14]         108,654         108,654
   TransformerBlock-15        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-16        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-17        [1, 160, 7, 7]         293,410         293,410
   TransformerBlock-18        [1, 320, 7, 7]       1,149,398       1,149,398
   TransformerBlock-19       [1, 1280, 7, 7]      18,087,512      18,087,512
             Linear-20               [1, 32]          40,992          40,992
=============================================================================
Total params: 20,759,318
Trainable params: 20,759,318
Non-trainable params: 0
-----------------------------------------------------------------------------
		 Training: Epoch(0) - Loss: 1.7345, Acc: 52.3349
		 Validation(0) - Loss: 1.3374, Acc: 60.2941
		 Training: Epoch(1) - Loss: 0.5159, Acc: 82.7697
		 Validation(1) - Loss: 0.2270, Acc: 91.1765
		 Training: Epoch(2) - Loss: 0.3618, Acc: 87.9227
		 Validation(2) - Loss: 0.2060, Acc: 91.1765
		 Training: Epoch(3) - Loss: 0.2802, Acc: 91.1433
		 Validation(3) - Loss: 0.2835, Acc: 94.1176
		 Training: Epoch(4) - Loss: 0.1626, Acc: 95.3301
		 Validation(4) - Loss: 0.4039, Acc: 88.2353
		 Training: Epoch(5) - Loss: 0.0902, Acc: 96.9404
		 Validation(5) - Loss: 0.3060, Acc: 92.6471
		 Training: Epoch(6) - Loss: 0.0513, Acc: 97.9066
		 Validation(6) - Loss: 0.2663, Acc: 92.6471
		 Training: Epoch(7) - Loss: 0.1204, Acc: 97.2625
		 Validation(7) - Loss: 0.2030, Acc: 91.1765
		 Training: Epoch(8) - Loss: 0.0790, Acc: 97.7456
		 Validation(8) - Loss: 0.2198, Acc: 88.2353
		 Training: Epoch(9) - Loss: 0.1234, Acc: 96.9404
		 Validation(9) - Loss: 0.2579, Acc: 89.7059
		 Training: Epoch(10) - Loss: 0.0595, Acc: 97.4235
		 Validation(10) - Loss: 0.3244, Acc: 89.7059
		 Training: Epoch(11) - Loss: 0.0823, Acc: 98.3897
		 Validation(11) - Loss: 0.2094, Acc: 95.5882
		 Training: Epoch(12) - Loss: 0.0308, Acc: 98.7118
		 Validation(12) - Loss: 0.0768, Acc: 95.5882
		 Training: Epoch(13) - Loss: 0.0391, Acc: 98.7118
		 Validation(13) - Loss: 0.1223, Acc: 97.0588
		 Training: Epoch(14) - Loss: 0.0403, Acc: 98.7118
		 Validation(14) - Loss: 0.2438, Acc: 91.1765
Finished.
Total time per fold: 654.7385499477386 seconds.
'''